# <div align=center>Introduction to Machine Learning with Spark</div>

## Spark

Spark is an open-source framework for fast and scalable data processing. It has built-in modules and libraries for machine learning, SQL and graph processing. Spark can run on multiple nodes using three cluster management technologies:

1. Spark standalone
2. YARN (used by the Hadoop ecosystem)
3. Mesos

This example uses a Spark standalone cluster running 1 master and 6 slave nodes. It has 24 cores and 17.2 GB of usable memory for processing.

![](cluster-overview.png)

## HDFS (Hadoop Distributed File System)

For storage, this Spark cluster uses HDFS, a scalable and fault-tolerant distributed file system that's used extensively in Hadoop applications. HDFS partitions files into blocks of fixed size (usually 128 or 256 MB) and replicates them across the cluster for high availability. Files can be put in HDFS through the command line or code. To read files, we can use the HDFS URI for that cluster, followed by the file path: *hdfs://hdfs-master-ip:port/path-to-file/*

The HDFS cluster in this example is running 1 master *(NameNode)* and 3 slave nodes *(DataNodes)*.

![](hdfsarchitecture.gif)

## Linking Python to the Spark cluster

In [21]:
import os, sys
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))

import pyspark
from pyspark import SparkContext, SparkConf

try:
    sc
except:
    pass ;
else:
    sc.stop()

conf = SparkConf().setMaster("spark://10.0.3.70:7077").setAppName("Testing Anchal").set("spark.driver.port",8200)
sc = SparkContext(conf=conf)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/home/ubuntu/spark/spark-1.4.1-bin-hadoop2.6/python/lib/py4j-0.8.2.1-src.zip/py4j/java_gateway.py", line 425, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server

## Working with data in Spark


A Resilient Distributed Dataset (RDD) is the basic data abstraction in Spark. It represents a collection of data elements that can be operated upon. 